# **Feature engineering of ENEM 2022 microdata**

In [13]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Loads the data from the previous stage's file
file_path = '../data/df_encoded.csv'
df = pd.read_csv(file_path, sep=';', encoding='iso-8859-1')
df

,TP_FAIXA_ETARIA,Q002,Q005,Q006,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO,TP_PRESENCA_LC,...,TP_ESCOLA_Public,TP_COR_RACA_Asian,TP_COR_RACA_Black,TP_COR_RACA_Indigenous,TP_COR_RACA_Mixed (Pardo),TP_COR_RACA_White,IN_TREINEIRO_Yes,Q022_Yes,Q024_Yes,Q025_Yes
0,12,6,3,16,NaN,NaN,NaN,NaN,NaN,Absent,...,0,0,1,0,0,0,0,0,0,0
1,12,1,3,0,NaN,NaN,NaN,NaN,NaN,Absent,...,0,0,0,0,0,1,0,1,1,1
2,3,2,2,16,498.8,546.0,421.1,565.3,760.0,Present,...,0,0,1,0,0,0,0,1,0,1
3,4,6,3,15,357.8,388.6,490.7,416.0,320.0,Present,...,0,0,0,0,1,0,0,1,1,1
4,2,5,2,16,NaN,NaN,NaN,NaN,NaN,Absent,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476103,1,4,4,8,NaN,NaN,NaN,NaN,NaN,Absent,...,1,0,0,0,0,1,0,1,0,1
3476104,12,6,3,16,NaN,NaN,NaN,NaN,NaN,Absent,...,1,0,0,0,0,1,0,1,1,1
3476105,0,0,4,9,583.3,627.0,527.9,637.1,660.0,Present,...,1,0,0,0,0,1,0,1,1,1
3476106,1,2,5,7,NaN,NaN,NaN,NaN,NaN,Absent,...,0,0,0,0,1,0,0,1,1,1


## **Feature Combination**

In [14]:
df_eng = df.copy()
del df

# Calculates the average of objective scores and discards individual columns
df_eng['objective_average'] = df_eng[['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT']].mean(axis=1)
df_eng['overall_average'] = df_eng[['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1)
df_eng.drop(columns=['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT'], inplace=True)

# Combines attendance information (if missed any exam receives 1, otherwise 0)
df_eng['absent'] = df_eng[['TP_PRESENCA_LC', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_MT']].apply(
    lambda row: any(value in ['Absent', 'Eliminated'] for value in row), axis=1
).astype(int)
df_eng.drop(columns=['TP_PRESENCA_LC', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_MT'], inplace=True)

# Combines race columns into two categories: white_asian and black_pardo_indigenous
df_eng['white_asian'] = (df_eng['TP_COR_RACA_White'] + df_eng['TP_COR_RACA_Asian']).clip(0, 1)
df_eng['black_pardo_indigenous'] = (df_eng['TP_COR_RACA_Black'] + df_eng['TP_COR_RACA_Mixed (Pardo)'] + df_eng['TP_COR_RACA_Indigenous']).clip(0, 1)
df_eng.drop(columns=['TP_COR_RACA_White', 'TP_COR_RACA_Asian', 'TP_COR_RACA_Black', 'TP_COR_RACA_Mixed (Pardo)', 'TP_COR_RACA_Indigenous', 'TP_ESCOLA_Public'], inplace=True)

# Uses only the 'has computer?' column
df_eng['has_computer'] = df_eng['Q024_Yes']
df_eng.drop(columns=['Q022_Yes', 'Q024_Yes', 'Q025_Yes'], inplace=True)

# Renames columns, removing prefixes and suffixes, converting to lowercase
rename_map = {
    'NU_NOTA_REDACAO': 'essay',
    'TP_ESCOLA_Private': 'private_school',
    'TP_SEXO_Female': 'female_gender',
    'IN_TREINEIRO_Yes': 'trainee',
    'Q002': 'mother_education',
    'Q005': 'family_size',
    'Q006': 'family_income',
    'TP_FAIXA_ETARIA': 'age_range'
}
df_eng.rename(columns=rename_map, inplace=True)
df_eng.columns = df_eng.columns.str.lower()
df_eng

,age_range,mother_education,family_size,family_income,essay,female_gender,private_school,trainee,objective_average,overall_average,absent,white_asian,black_pardo_indigenous,has_computer
0,12,6,3,16,NaN,0,0,0,NaN,NaN,1,0,1,0
1,12,1,3,0,NaN,0,0,0,NaN,NaN,1,1,0,1
2,3,2,2,16,760.0,1,0,0,507.800,558.24,0,0,1,0
3,4,6,3,15,320.0,0,0,0,413.275,394.62,0,0,1,1
4,2,5,2,16,NaN,0,0,0,NaN,NaN,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476103,1,4,4,8,NaN,0,0,0,NaN,NaN,1,1,0,0
3476104,12,6,3,16,NaN,1,0,0,NaN,NaN,1,1,0,1
3476105,0,0,4,9,660.0,0,0,0,593.825,607.06,0,1,0,1
3476106,1,2,5,7,NaN,0,1,0,NaN,NaN,1,0,1,1


## **Considerations on arbitrary weighting**

The columns for family income, age group, and mother's education represent value ranges rather than continuous numbers. Since each range covers an interval, **there is a degree of arbitrary weighting when treating them as numerical values**. This occurs because the intervals do not necessarily have the same width, and the difference between ranges may not be linear. However, this approach simplifies modeling and avoids the dimensional explosion that would occur with One Hot Encoding. **This simpler model was chosen to maintain interpretability and computational efficiency.**

In [10]:
# Values for the 'renda_familiar' (family_income) column:
#  0 → No income
#  1 → Up to R$ 1,212.00
#  2 → From R$ 1,212.01 to R$ 1,818.00
#  3 → From R$ 1,818.01 to R$ 2,424.00
#  4 → From R$ 2,424.01 to R$ 3,030.00
#  5 → From R$ 3.030,01 to R$ 3,636.00
#  6 → From R$ 3,636.01 to R$ 4,848.00
#  7 → From R$ 4,848.01 to R$ 6,060.00
#  8 → From R$ 6,060.01 to R$ 7,272.00
#  9 → From R$ 7,272.01 to R$ 8,484.00
# 10 → From R$ 8,484.01 to R$ 9,696.00
# 11 → From R$ 9,696.01 to R$ 10,908.00
# 12 → From R$ 10,908.01 to R$ 12,120.00
# 13 → From R$ 12,120.01 to R$ 14,544.00
# 14 → From R$ 14,544.01 to R$ 18,180.00
# 15 → From R$ 18,180.01 to R$ 24,240.00
# 16 → Over R$ 24,240.00

# Values for the 'faixa_etaria' (age_group) column:
#  0 → Under 17 years old
#  1 → 17 years old
#  2 → 18 years old
#  3 → 19 years old
#  4 → 20 years old
#  5 → 21 years old
#  6 → 22 years old
#  7 → 23 years old
#  8 → 24 years old
#  9 → 25 years old
# 10 → Between 26 and 30 years old
# 11 → Between 31 and 35 years old
# 12 → Between 36 and 40 years old
# 13 → Between 41 and 45 years old
# 14 → Between 46 and 50 years old
# 15 → Between 51 and 55 years old
# 16 → Between 56 and 60 years old
# 17 → Between 61 and 65 years old
# 18 → Between 66 and 70 years old
# 19 → Over 70 years old

# Values for the 'escolaridade_mae' (mothers_education) column:
# 0 → Never studied.
# 1 → Did not complete 4th grade/5th year of Elementary School.
# 2 → Completed 4th grade/5th year, but did not complete 8th grade/9th year of Elementary School.
# 3 → Completed 8th grade/9th year of Elementary School, but did not complete High School.
# 4 → Completed High School, but did not complete College.
# 5 → Completed College, but did not complete Graduate School.
# 6 → Completed Graduate School.
# 7 → Unknown - was replaced by the column's mode

In [15]:
# Calculates thresholds for binary classification, considering the mean plus one standard deviation
objective_limit = df_eng['objective_average'].mean() + df_eng['objective_average'].std()
essay_limit = df_eng['essay'].mean() + df_eng['essay'].std()
overall_limit = df_eng['overall_average'].mean() + df_eng['overall_average'].std()

print(f'Threshold for objective average to be considered high: {round(objective_limit, 2)}')
print(f'Threshold for essay score to be considered high: {round(essay_limit, 2)}')
print(f'Threshold for overall average to be considered high: {round(overall_limit, 2)}')

# Creates binary variables
df_eng['high_objective_average'] = (df_eng['objective_average'] > objective_limit).astype(int)
df_eng['high_essay_score'] = (df_eng['essay'] > essay_limit).astype(int)
df_eng['high_overall_average'] = (df_eng['overall_average'] > overall_limit).astype(int)

# Removes original score columns
df_eng.drop(columns=['objective_average', 'essay', 'overall_average'], inplace=True)
df_eng

Threshold for objective average to be considered high: 593.73
Threshold for essay score to be considered high: 830.69
Threshold for overall average to be considered high: 630.96


,age_range,mother_education,family_size,family_income,female_gender,private_school,trainee,absent,white_asian,black_pardo_indigenous,has_computer,high_objective_average,high_essay_score,high_overall_average
0,12,6,3,16,0,0,0,1,0,1,0,0,0,0
1,12,1,3,0,0,0,0,1,1,0,1,0,0,0
2,3,2,2,16,1,0,0,0,0,1,0,0,0,0
3,4,6,3,15,0,0,0,0,0,1,1,0,0,0
4,2,5,2,16,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476103,1,4,4,8,0,0,0,1,1,0,0,0,0,0
3476104,12,6,3,16,1,0,0,1,1,0,1,0,0,0
3476105,0,0,4,9,0,0,0,0,1,0,1,1,0,0
3476106,1,2,5,7,0,1,0,1,0,1,1,0,0,0


In [16]:
# Saves the dataframe for the next stage (modeling)
df_eng.to_csv('../data/df_eng.csv', sep=';', encoding='iso-8859-1', index=False)